In [1]:
from gurobipy import *
import sys
import time
import os
import numpy as np

In [2]:
def Preprocess(file):
    array = []
    for line in file:
        array.append(line)
    tokens = array[4].split()
    vehicle = int(tokens[0])
    Capacity = int(tokens[1])
    xc={}
    yc={}
    demand={}
    ready={}
    due={}
    service={}
    for i in range(9,len(array)):
        if len(array[i].split())>6:
            id,x,y,d,red,du,serv=array[i].split()
            id=int(id)
            xc[id]=int(x)
            yc[id]=int(y)
            demand[id]=int(d)
            ready[id]=int(red)
            due[id]=int(du)
            service[id]=int(serv)
    return vehicle, Capacity, xc, yc, demand, ready, due, service

In [3]:
def SolveVRPTW_LR(vehicle, Capacity, xc, yc, demand, ready, due, service, M):
    master = Model("Master Problem")
    subproblem = Model("Route Generator Subproblem")
    
    ##############################################################################################################
    ##########       information for the fisrt iteration of the master problem         ###########################
    routeindex = 0 # first path, this is infeasible for the subproblem but feasible for the master    
    routecost = {}
    Adj = {} # adjacency matrix of the nodes
    for i in range(len(xc)):
        for j in range(i+1,len(xc)):
            Adj[i,j] = pow((pow(xc[i]- xc[j], 2) + pow(yc[i] - yc[j], 2)), 0.5)
            Adj[j,i] = Adj[i,j]
    a = {} # a[i, routeindex] = 1 if the route visits vertex i, 0 otherwise
    for i in range(len(xc)):
        a[i, routeindex] = 1 # this is a route visiting all the vertices, not feasible to subproblem.
    routecost[routeindex] = sys.maxsize ## the cost of such a route is large 
    mv={} ### master variable indexed by routes
    mv[routeindex] = master.addVar(vtype=GRB.CONTINUOUS) ## we are solving linear relaxation
    #master constraint generation
    mastercons={}
    mastercons[0]=master.addConstr(mv[routeindex] <= vehicle) #number of vehicles
    for i in range(1,len(xc)):
        mastercons[i]=master.addConstr(a[i,routeindex]*mv[routeindex]==1) # all the customers need to be visited    
    master.setObjective(routecost[routeindex]*mv[routeindex],GRB.MINIMIZE) 
    master.optimize()
    
    #################################################################################################################
    ############################                       The Subproblem                        ########################
    w={} ## subproblem decision var
    s={} ## subproblem decision var 
    for i in range(len(xc)):
        for j in range(i+1,len(xc)):
            w[i,j]=subproblem.addVar(vtype=GRB.BINARY) #subproblem variables generation
            w[j,i]=subproblem.addVar(vtype=GRB.BINARY) 
        s[i]=subproblem.addVar(vtype=GRB.CONTINUOUS) #subproblem variables generation
    #subproblem constraints
    #1. Leaving from the depot
    subproblem.addConstr(quicksum(w[0,j] for j in range(len(xc)) if j!=0)==1)
    #2. Flow constraint
    for i in range(len(xc)):
        subproblem.addConstr(quicksum(w[i,j]-w[j,i] for j in range(len(xc)) if i!=j)==0)
    #3. capacity constraint
    subproblem.addConstr(quicksum(demand[i]*w[i,j] for i in range(len(xc)) for j in range(len(xc)) if i!=j)<=Capacity)
    #4. Subtour elimination (Time window)
    for i in range(len(xc)):
        for j in range(len(xc)):
            if i!=j and j!=0:
                subproblem.addConstr(s[i]+Adj[i,j]+service[i]-M*(1-w[i,j])<=s[j])
        if i!=0:
            subproblem.addConstr(s[i]+Adj[i,0]+service[i]-max(abs(ready[i]-due[j]),abs(ready[j]-due[i]))*(1-w[i,0])<=due[0])
    #5. Time window
    for i in range(len(xc)):
        subproblem.addConstr(s[i]>=ready[i])
        subproblem.addConstr(s[i]<=due[i])    
    
    #########################################################################################################################
    ################        Initialization of variables  and getting ready for column generation      #######################
    flag=True
    iteration=0
    reducedcost={}
    reducedcost[0]=0
    masterobjectives={}
    masterobjectives[routeindex]=master.objVal    
    Routes = {}
    ###########################################################################################################################
    #############################                     Column Generation                               #########################
    while(flag):
        ### extract the dual of the master ###
        dual={}
        for i in range(len(xc)):
            dual[i] = mastercons[i].pi
        subproblem.setObjective(quicksum((Adj[i,j]-mastercons[i].pi)*w[i,j] for i in range(len(xc)) for j in range(len(xc)) if i!=j),GRB.MINIMIZE)
        subproblem.optimize()
        ##################   optimality condition #################
        if round(subproblem.objVal,2) >= 0:
            flag = False
        ####################   Creation of the new routes   ######################  
        edges={}
        ctr=0
        for i in range(len(xc)):
            for j in range(len(xc)):
                if (i!=j and w[i,j].x > 0.9):
                    edges[i,j] = 1
                    ctr = ctr + 1
                else:
                    edges[i,j] = 0
        route = "0"
        f = True
        current = 0
        while f:
            for j in range(len(xc)):
                if (current != j and edges[current, j] == 1):
                    route = route + "-> " + str(j)
                    if (j != 0):
                        current = j
                        break
                    else:
                        f = False
                
        # calculate the costot = cost of the path generated by the subproblem
        costot=0
        for i in range(len(xc)):
            for j in range(len(xc)):
                if i!=j and w[i,j].x>0.9:
                    costot = costot+Adj[i,j]
        routeindex = len(routecost) ## update the new route index
        routecost[routeindex] = costot 
        reducedcost[routeindex] = subproblem.objVal 
        # update a[i,r] according to the new path generated by the subproblem
        a[0,routeindex] = 1
        for i in range(1, len(xc)):
            a[i,routeindex] = sum(w[i,j].x for j in range(len(xc)) if i!=j)
        # now add the variable or column with the corresponding a[i,r] to the master
        col = Column()
        for i in range(len(xc)):
            col.addTerms(a[i,routeindex], mastercons[i])
        mv[routeindex] = master.addVar(obj = routecost[routeindex], vtype=GRB.CONTINUOUS, column=col)
        Routes[routeindex] = route
        master.update()
        master.optimize()
        masterobjectives[routeindex]=master.objVal
        iteration+=1
        print("**********************")   
    master.optimize()
    
    ########################################################################################################################
    ############################               Early Branching for 0/1                             #########################   
    print("*************************************************************************************************")
    print("*************************************************************************************************")
    print("Early Branching")
    print("The routes are: ")
    intobj = 0
    for r in range(len(routecost)):
        if(mv[r].X >= 0.1):
            intobj = intobj + routecost[r]
            print(Routes[r])
            print('\n')
    print("The Optimal Integer Objective is: " + str(intobj))

In [4]:
file = open("C:\\Users\\Prasangsha\\Downloads\\C101.txt", 'r')
vehicle, Capacity, xc, yc, demand, ready, due, service = Preprocess(file)
M = 9999 #big M value to be used in a subproblem constraint
SolveVRPTW_LR(vehicle, Capacity, xc, yc, demand, ready, due, service, M)

Using license file C:\Users\Prasangsha\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 26 rows, 1 columns and 26 nonzeros
Model fingerprint: 0xb48eb330
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+18, 9e+18]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1 rows and 26 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.2233720e+18   0.000000e+00   2.500000e-05      0s
       0    9.2233720e+18   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  9.223372037e+18
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 730 rows, 676 columns and 3927 nonzeros
Model fin

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 9e+18]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.8446744e+49   4.500000e+31   1.844674e+19      0s
       2    9.2233720e+18   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  9.223372037e+18
**********************
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 730 rows, 676 columns and 3927 nonzeros
Model fingerprint: 0x6d6c084d
Variable types: 26 continuous, 650 integer (650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e-01, 9e+18]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
      


Root relaxation: cutoff, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      -9.223e+18 -9.223e+18  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.07 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: -9.22337e+18 7.10543e-15 
No other solutions better than -9.22337e+18

Optimal solution found (tolerance 1.00e-04)
Best objective -9.223372036855e+18, best bound -9.223372036855e+18, gap 0.0000%
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 26 rows, 9 columns and 61 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 9e+18]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Iteration    Objective

         to avoid numerical issues.

MIP start from previous solve produced solution with objective -6.21725e-14 (0.01s)
Loaded MIP start from previous solve with objective -6.21725e-14

Presolve removed 688 rows and 540 columns
Presolve time: 0.02s
Presolved: 42 rows, 136 columns, 467 nonzeros
Found heuristic solution: objective -44.7411515
Variable types: 11 continuous, 125 integer (125 binary)

Root relaxation: objective -9.223372e+18, 39 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    -9.22337e+18 -9.223e+18  0.00%     -    0s

Explored 0 nodes (39 simplex iterations) in 0.05 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: -9.22337e+18 -44.7412 -6.21725e-14 
No other solutions better than -9.22337e+18

Optimal solution found (tolerance 1.00e-04)
Best objective -9.223372036855e+18, best bound -9.223

       0   -9.2233720e+48   1.433333e+31   9.223372e+18      0s
       4    9.2233720e+18   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  9.223372037e+18
**********************
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 730 rows, 676 columns and 3927 nonzeros
Model fingerprint: 0x2af5b46d
Variable types: 26 continuous, 650 integer (650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e-02, 9e+18]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

MIP start from previous solve produced solution with objective -1.77636e-15 (0.01s)
Loaded MIP start from previous solve with objective -1.77636e-15

Presolve removed 674 rows and 493 columns
Presolve time: 0.03s
Presolved: 56 rows, 183 columns, 758 nonzeros
Variable types: 14 continuous, 169 integer

...

Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e-02, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start from previous solve produced solution with objective 4.1308 (0.01s)
Loaded MIP start from previous solve with objective 4.1308

Presolve removed 673 rows and 439 columns
Presolve time: 0.02s
Presolved: 57 rows, 237 columns, 1019 nonzeros
Variable types: 14 continuous, 223 integer (223 binary)

Root relaxation: objective -1.016575e+01, 44 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -10.16575    0   15    4.13080  -10.16575   346%     -    0s
H    0     0                       0.0000000  -10.16575      -     -    0s
H    0     0                      -7.5458065   -9.98083  32.3%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 1
  Implied bound: 1
  MIR: 7
  Zero half: 15
  R

  Objective range  [3e+01, 9e+18]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.1648759e+30   1.937500e+30   5.164876e+00      0s
       1    1.9181362e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.918136198e+02
**********************
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 730 rows, 676 columns and 3927 nonzeros
Model fingerprint: 0xe20f526a
Variable types: 26 continuous, 650 integer (650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e-01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start from previous solve produced solution with objective 0 (0.01s)
Loaded MIP start from previous solve with objective 0

Presolv


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -12.91068    0   12    7.00439  -12.91068   284%     -    0s
H    0     0                      -3.7571616  -12.91068   244%     -    0s
H    0     0                      -9.8962892  -12.91068  30.5%     -    0s

Cutting planes:
  Gomory: 1
  Implied bound: 1
  MIR: 3
  Mod-K: 1
  RLT: 1

Explored 1 nodes (47 simplex iterations) in 0.05 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: -9.89629 -3.75716 7.00439 
No other solutions better than -9.89629

Optimal solution found (tolerance 1.00e-04)
Best objective -9.896289166278e+00, best bound -9.896289166278e+00, gap 0.0000%
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 26 rows, 142 columns and 1181 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 9e+18]
  Bounds range     [0e

Optimize a model with 730 rows, 676 columns and 3927 nonzeros
Model fingerprint: 0x85cbc795
Variable types: 26 continuous, 650 integer (650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [6e-02, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start from previous solve produced solution with objective 0 (0.01s)
Loaded MIP start from previous solve with objective 0

Presolve removed 669 rows and 430 columns
Presolve time: 0.02s
Presolved: 61 rows, 246 columns, 1091 nonzeros
Variable types: 14 continuous, 232 integer (232 binary)

Root relaxation: objective -9.875994e+00, 42 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -9.87599    0   10    0.00000   -9.87599      -     -    0s
     0     0   -4.06564    0   17    0.00000   -4.06564      -     -    0s
     0     0   -4.06564 


Cutting planes:
  Gomory: 7
  Implied bound: 3
  MIR: 4
  StrongCG: 1
  Mod-K: 4
  RLT: 8

Explored 1 nodes (114 simplex iterations) in 0.08 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: -1.77636e-15 5.32907e-15 

Optimal solution found (tolerance 1.00e-04)
Best objective -1.776356839400e-15, best bound -1.776356839400e-15, gap 0.0000%
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 26 rows, 149 columns and 1226 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 9e+18]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9181362e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.918136198e+02
**********************
Gurobi Optimizer versi